# Sklearn

In [1]:
from topic_modeling_functions import *
from transliterate import translit
from tqdm import tqdm_notebook

pd.set_option('display.max_colwidth', -1)

In [2]:
start = 5  # min n_topics
stop = 21  # max n_topics + 1
step = 1
n_top_words = 10

In [12]:
count_vect = CountVectorizer(lowercase=False, tokenizer=lambda r: r)
tfidf_vect = TfidfVectorizer(lowercase=False, tokenizer=lambda r: r)

### Пример с текстами А. Волкова

In [4]:
df_volkov = pd.read_pickle('authors.pkl').iloc[0]

In [5]:
%%time

count_data, count_feature_names = get_data_vectorized(count_vect, df_volkov.text_pymystem_list)
tfidf_data, tfidf_feature_names = get_data_vectorized(tfidf_vect, df_volkov.text_pymystem_list)

CPU times: user 46.4 ms, sys: 0 ns, total: 46.4 ms
Wall time: 51.4 ms


In [6]:
%%time

# nmf падает, если n_topics > числа документов коллекции

lda_res = get_models_for_n_topics('lda', count_data, start, stop, step)
nmf_res = get_models_for_n_topics('nmf', tfidf_data, start, stop, step)

CPU times: user 1.74 s, sys: 312 ms, total: 2.05 s
Wall time: 1.72 s


In [7]:
%%time

lda_topics = get_n_top_words(lda_res, count_feature_names, n_top_words)
nmf_topics = get_n_top_words(nmf_res, tfidf_feature_names, n_top_words)

CPU times: user 28.5 ms, sys: 12.2 ms, total: 40.7 ms
Wall time: 25.3 ms


In [8]:
df_volkov_results = pd.DataFrame({'lda': lda_topics}, index=get_index(start, stop, step))
df_volkov_results['nmf'] = pd.Series(nmf_topics, index=get_index(start, tfidf_data.shape[0] + 1, step))

In [9]:
df_volkov_results.loc['n_topics_5']

,lda,nmf
n_topic_1,"[заснуть, урфин, страна, волшебный, марран, рыба, просить, каждый, давать, звать]","[страна, урфин, марран, город, изумрудный, волшебный, элли, помощь, страшило, джюс]"
n_topic_2,"[страна, элли, урфин, волшебный, город, страшило, изумрудный, помощь, железный, гудвин]","[элли, гудвин, девочка, бастинд, лев, дровосек, волшебница, железный, путешественник, фургон]"
n_topic_3,"[урфин, страна, марран, тим, волшебный, карфакс, изумрудный, становиться, энни, джюс]","[генерал, энни, гном, тим, страшило, похищать, камень, совет, изучать, привозить]"
n_topic_4,"[страна, король, правитель, марран, элли, время, ребенок, испугаться, руф, билан]","[ящик, канзас, капитан, карра, карфакс, качество, кирпич, клетка, клоун, клюв]"
n_topic_5,"[страна, король, подземный, время, вода, элли, житель, город, тотошка, помощь]","[колдунья, гном, чарли, книга, страна, волшебный, билан, руф, дракон, тилль]"


In [10]:
df_volkov_results.loc[('n_topics_6', 'n_topic_5')]

lda    [страна, король, подземный, время, вода, город, должный, житель, элли, железный]
nmf    [колдунья, гном, чарли, страна, книга, билан, руф, волшебный, дракон, тилль]    
Name: (n_topics_6, n_topic_5), dtype: object

In [11]:
df_volkov_results.to_csv('topic_modeling/volkov.tsv', sep='\t')

### Для каждой литературной традиции:

In [12]:
df_traditions = pd.read_pickle('traditions_topic_modeling.pkl')

In [13]:
%%time

traditions_topics = {}

for i in tqdm_notebook(df_traditions.index):
    tradition = df_traditions.loc[i, 'tradition']
    res = get_words_for_topics(df_traditions.loc[i, 'text_pymystem_list'], count_vect, tfidf_vect,
                                                                   start, stop, step, n_top_words)
    traditions_topics[tradition] = res
    res.to_csv('topic_modeling/traditions/{}.tsv'.format(translit(tradition.replace(' ', '_'), 'ru',
                                                                          reversed=True)), sep='\t')

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


CPU times: user 15min 6s, sys: 15min 51s, total: 30min 57s
Wall time: 12min 43s


In [14]:
traditions_topics.keys()

dict_keys(['Американская литература', 'Античная литература', 'Британская литература', 'Прочая восточная литература', 'Европейская литература', 'Скандинавская литература', 'Прочая европейская литература', 'Латиноамериканская литература', 'Русская литература', 'Прочая литература'])

In [15]:
traditions_topics['Европейская литература'].loc['n_topics_10']

,lda,nmf
n_topic_1,"[человек, становиться, время, отец, жизнь, мочь, друг, сын, дом, девушка]","[человек, жизнь, становиться, друг, время, отец, мочь, дом, сын, год]"
n_topic_2,"[давид, саул, брут, пан, адам, цезарь, шпиль, ионафан, авенир, мюнхаузен]","[дон, хуан, донья, луис, карлос, мануэль, родриго, жуан, педро, кихот]"
n_topic_3,"[фауст, мочь, отец, человек, дом, жизнь, время, становиться, жена, друг]","[принц, принцесса, король, курфюрст, фернандо, эмилия, герцог, турандот, дженнаро, граф]"
n_topic_4,"[дон, дом, время, становиться, год, хуан, мочь, человек, друг, день]","[де, госпожа, вальмон, манон, жюльен, маркиза, эжени, шевалье, мадемуазель, г]"
n_topic_5,"[галилей, гленарывать, айртон, андреа, дункан, грант, человек, людовико, время, мочь]","[маркиз, маркиза, флориндо, беатриче, кавалер, мария, панталоне, синьора, фабрицио, чекко]"
n_topic_6,"[время, человек, дом, день, становиться, мочь, сын, жизнь, отец, однако]","[генрих, король, матильда, екатерина, наваррский, королева, марго, колокол, жанна, гиз]"
n_topic_7,"[становиться, человек, время, жизнь, де, день, мочь, год, граф, жена]","[эмма, мистер, дэвид, шарль, родольф, элтон, фрэнк, леон, лера, флора]"
n_topic_8,"[шляпа, надевать, желтый, метод, обсуждение, человек, дискуссия, шесть, зеленый, группа]","[цезарь, клеопатра, антиох, тит, брут, нерон, рим, царица, римский, император]"
n_topic_9,"[друг, граф, жизнь, человек, становиться, графиня, время, пан, сын, год]","[граф, сюзанна, фигаро, графиня, керубиный, леон, розин, бартоло, базиль, марселина]"
n_topic_10,"[немо, лот, магомет, гофман, нед, сеид, ткач, ансельмо, человек, омар]","[робинзон, остров, испанец, корабль, пятница, дикарь, капитан, посев, леон, англия]"


### Для каждой литературы:

In [8]:
df_countries = pd.read_pickle('countries_topic_modeling.pkl')

In [13]:
%%time

countries_topics = {}

for i in tqdm_notebook(df_countries.index):
    country = df_countries.loc[i, 'country']
    res = get_words_for_topics(df_countries.loc[i, 'text_pymystem_list'], count_vect, tfidf_vect,
                                                                   start, stop, step, n_top_words)
    countries_topics[country] = res
    res.to_csv('topic_modeling/countries/{}.tsv'.format(translit(country.replace(' ', '_'), 'ru',
                                                                        reversed=True)), sep='\t')

HBox(children=(IntProgress(value=0, max=45), HTML(value='')))


CPU times: user 13min 17s, sys: 14min 31s, total: 27min 48s
Wall time: 8min 48s


In [14]:
countries_topics.keys()

dict_keys(['Австрийская', 'Швейцарская', 'Британская', 'Австралийская', 'Белорусская', 'Польская', 'Чешская', 'Советская', 'Норвежская', 'Шведская', '19 век', 'Древнеримская', 'Аргентинская', 'Французская', 'Немецкая', '18 век', 'Датская', 'Современная', 'Португальская', 'Итальянская', 'Еврейская', 'Чилийская', 'Древность', 'Бельгийская', 'Исландская', 'Шотландская', 'Персидская', 'Финская', 'Колумбийская', 'Кубинская', 'Нидерландская', 'Древнегреческая', 'Турецкая', 'Грузинская', 'Китайская', 'Японская', 'США', 'Ирландская', 'Украинская', 'Гватемальская', 'Индийская', 'Бразильская', 'Армянская', 'Югославская', 'Испанская'])

In [15]:
countries_topics['Японская'].loc['n_topics_10']

,lda,nmf
n_topic_1,"[комако, симамура, симамур, кикудзь, йоко, девушка, мочь, учительница, юкико, фумико]","[государь, акэбоно, дворец, рукав, луна, платье, принц, отец, любовь, обитель]"
n_topic_2,"[тэнго, аомамэ, эри, лидер, усикава, отец, кокон, светлость, воздушный, видеть]","[овца, герой, крыса, маска, сэнсей, профессор, жена, секретарь, лицо, человек]"
n_topic_3,"[кикудзь, тиэко, мидзогути, девушка, храм, отец, фумико, становиться, женщина, кадзуко]","[кикудзь, фумико, юкико, тикако, оот, куримото, сан, чашка, таэко, госпожа]"
n_topic_4,"[такаси, брат, хотеть, жена, герой, комако, время, дом, симамура, лицо]","[песок, мужчина, тадземара, женщина, разбойник, яма, веревка, убивать, муж, лопата]"
n_topic_5,"[ящик, человек, врач, военный, записка, клиника, лжечеловек, девушка, писать, женщина]","[ватанабэ, наоко, рэйко, мидори, нагасава, кидзука, хацуми, поручик, девушка, приезжать]"
n_topic_6,"[девушка, кими, становиться, сэйкити, год, женщина, соноко, мужчина, человек, лицо]","[светлость, есихидэ, карета, обезьянка, картина, художник, рассказчица, мука, ширма, ад]"
n_topic_7,"[человек, женщина, становиться, дом, жизнь, год, девушка, мочь, мужчина, любовь]","[юити, сюнсукэ, кабурага, ясуко, юноша, кеко, нобутака, писатель, г, кавада]"
n_topic_8,"[юкико, таэко, сатико, сестра, дом, окубат, семья, год, асия, макиок]","[сюнкин, сасукэ, музыка, сюнс, мастер, ученик, учитель, учительница, ослепнуть, жизнеописание]"
n_topic_9,"[юити, герой, такаси, маска, жена, лицо, сюнкин, мицу, брат, сюнсукэ]","[ящик, человек, врач, лжечеловек, военный, клиника, записка, писать, вылезать, девушка]"
n_topic_10,"[мужчина, женщина, песок, яма, тадземара, веревка, хотеть, кикудзь, разбойник, убивать]","[мидзогути, храм, касивага, цурукав, настоятель, золотой, уико, досэн, офицер, мальчик]"


### Для авторов (6 и более текстов в корпусе):

In [16]:
df_authors = pd.read_pickle('authors.pkl')

In [17]:
%%time

authors_topics = {}

for i in tqdm_notebook(df_authors.index):
    author = df_authors.loc[i, 'author']
    res = get_words_for_topics(df_authors.loc[i, 'text_pymystem_list'], count_vect, tfidf_vect,
                                                                start, stop, step, n_top_words)
    authors_topics[author] = res
    res.to_csv('topic_modeling/authors/{}.tsv'.format(translit(author.replace(' ', '_'), 'ru',
                                                                    reversed=True)), sep='\t')

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))


CPU times: user 14.8 s, sys: 11.2 s, total: 26 s
Wall time: 13.1 s


In [18]:
authors_topics.keys()

dict_keys(['Еврипид', 'Вольтер', 'Волков', 'Расин', 'Корнель', 'Кальдерон'])

In [19]:
authors_topics['Расин'].loc['n_topics_6']

,lda,nmf
n_topic_1,"[нерон, британик, агриппина, цезарь, юния, бурра, император, мать, нарцисс, рим]","[пирр, орест, гермиона, андромах, андромаха, гектор, сын, убийство, говорить, убивать]"
n_topic_2,"[пирр, береника, решать, тит, орест, готовый, запад, думать, предполагать, просто]","[нерон, британик, агриппина, бурра, юния, император, цезарь, нарцисс, мать, юний]"
n_topic_3,"[храм, иодать, царь, гофолий, мальчик, левит, элиаким, готовый, нерон, иоас]","[ипполит, федр, энон, арикий, тесь, тесей, сын, афины, отец, власть]"
n_topic_4,"[ипполит, федр, береника, тит, сын, мочь, любовь, отец, арикий, хотеть]","[иодать, гофолий, храм, левит, элиаким, мальчик, иоас, священник, иудейский, царь]"
n_topic_5,"[береника, тит, антиох, император, нерон, народ, мочь, любовь, цезарь, римский]","[ифигения, ахилл, агамемнон, дочь, эрифил, клитемнестр, улисс, царь, жених, авлида]"
n_topic_6,"[пирр, ахилл, ифигения, орест, дочь, гермиона, царь, агамемнон, отец, сын]","[береника, тит, антиох, цезарь, император, закон, рим, римский, народ, мочь]"


### Для всей выборки:

In [20]:
df = pd.read_pickle('metatable_preprocessed.pkl')

In [21]:
%%time

df_results = get_words_for_topics(df.text_pymystem_list, count_vect, tfidf_vect, start, stop, step, n_top_words)

CPU times: user 11min 44s, sys: 12min 42s, total: 24min 27s
Wall time: 8min 17s


In [22]:
df_results.loc['n_topics_5']

,lda,nmf
n_topic_1,"[человек, время, становиться, мочь, жизнь, друг, отец, дом, год, день]","[человек, жизнь, герой, время, становиться, дом, друг, отец, год, мочь]"
n_topic_2,"[сальери, равик, моцарт, итен, вокульский, рентс, джонатан, калинович, тилем, неля]","[ваш, клиент, компания, продукт, пример, бизнес, сотрудник, человек, решение, использовать]"
n_topic_3,"[ширин, хосров, сиавуш, уинстон, хакон, таэко, кмицица, фритьоф, шах, колдуэлл]","[дон, хуан, жуан, донья, карлос, луис, мануэль, кихот, изабелла, родриго]"
n_topic_4,"[продукт, клиент, компания, уэсли, ваш, юити, рынок, дэвис, потребитель, бизнес]","[эдип, фивы, креонт, антигона, этеокла, иокастый, хор, царь, полиник, лаий]"
n_topic_5,"[иван, дон, становиться, мальчик, день, решать, идти, время, человек, дом]","[иван, царь, олеся, васильевич, конь, африканович, яга, иванович, жар, волк]"


In [23]:
df_results.to_csv('topic_modeling/all_data.tsv', sep='\t')

# Gensim

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [24]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [25]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(df['text_pymystem_list'], 
                               min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(df['text_pymystem_list'], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 59 s, sys: 561 ms, total: 59.5 s
Wall time: 59.7 s


In [26]:
trigram_mod[bigram_mod[df['text_pymystem_list'][2]]][0:20]

['лето',
 'последний',
 'главный_героиня',
 'аля',
 'амосов',
 'родной',
 'деревня',
 'летовка',
 'прошлый',
 'год',
 'похороны',
 'мать',
 'хотеть',
 'узнавать',
 'новость',
 'тетка',
 'анисья',
 'маня',
 'приезжать',
 'навещать']

In [27]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### Create the Dictionary and Corpus needed for Topic Modeling

In [28]:
%%time

id2word = corpora.Dictionary(df['text_pymystem_list'])

# Create Corpus
data_words_bigrams = make_bigrams(df['text_pymystem_list'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_words_bigrams]

CPU times: user 10.8 s, sys: 125 ms, total: 11 s
Wall time: 11 s


In [29]:
corpus[:1]

[[(0, 13),
  (1, 22),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 5),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 3),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 4),
  (24, 3),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (30, 4),
  (31, 1),
  (32, 3),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 2),
  (41, 3),
  (42, 4),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 2),
  (47, 2),
  (48, 7),
  (49, 2),
  (50, 2),
  (51, 1),
  (52, 2),
  (54, 1),
  (55, 3),
  (56, 2),
  (57, 3),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 3),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 2),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 2),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 3),
  (78, 1),
  (80, 1),
  (82, 1),
  (83, 1),
  (84, 2),
  (85, 2),
  (86, 1),
  (87, 2),
  (88, 3),
  (89, 1),
  (90, 1),
  (91, 1),
  (92, 1),
  (93, 1),
  (94, 1),
  (95, 1),
  (96,

In [30]:
id2word[50]

'девушка'

In [31]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('анкудин', 13),
  ('анюта', 22),
  ('балалайка', 1),
  ('бежать', 1),
  ('безделье', 1),
  ('бесплатенмельник', 1),
  ('благодарить', 1),
  ('благословлять', 1),
  ('боярин', 1),
  ('брифли', 1),
  ('вдали', 1),
  ('велеть', 5),
  ('верить', 1),
  ('вернуться', 1),
  ('весть', 1),
  ('ветер', 1),
  ('видение', 1),
  ('видеть', 3),
  ('вино', 1),
  ('внешне', 1),
  ('возвращение', 1),
  ('вокруг', 1),
  ('вопрос', 4),
  ('ворожба', 3),
  ('ворожить', 1),
  ('ворота', 1),
  ('вперед', 1),
  ('вращать', 1),
  ('встречать', 4),
  ('вступать', 1),
  ('выбор', 3),
  ('вызывать', 1),
  ('выносить', 1),
  ('выпивать', 1),
  ('выпроваживать', 1),
  ('выражать', 1),
  ('вырастать', 2),
  ('выслушивать', 1),
  ('выходить', 2),
  ('глаз', 3),
  ('говорить', 4),
  ('гоняться', 1),
  ('гулять', 1),
  ('давать', 1),
  ('давно', 2),
  ('двор', 2),
  ('дворянин', 7),
  ('дворянский', 2),
  ('девушка', 2),
  ('действие', 1),
  ('делать', 2),
  ('деньги', 1),
  ('деревня', 3),
  ('дерево', 2),
  ('доб

### Building the Topic Model

In [32]:
%%time

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 2min 50s, sys: 3min 19s, total: 6min 9s
Wall time: 1min 59s


### View the topics in LDA model

In [33]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"невеста" + 0.013*"жених" + 0.011*"свадьба" + 0.008*"михаил" + '
  '0.008*"сиавуш" + 0.006*"мужик" + 0.006*"татарский" + 0.006*"сунуть" + '
  '0.006*"петь" + 0.006*"сохраб"'),
 (1,
  '0.035*"эринния" + 0.014*"цезарь" + 0.013*"бенвенуто" + 0.012*"царь" + '
  '0.009*"тевье" + 0.008*"рим" + 0.008*"хакон" + 0.007*"тарелкин" + '
  '0.006*"варравин" + 0.006*"тилем"'),
 (2,
  '0.015*"князь" + 0.015*"петр" + 0.013*"иван" + 0.012*"григорий" + '
  '0.011*"солдат" + 0.009*"русский" + 0.008*"армия" + 0.008*"генерал" + '
  '0.007*"война" + 0.005*"офицер"'),
 (3,
  '0.006*"отправляться" + 0.006*"человек" + 0.006*"увидеть" + 0.005*"решать" + '
  '0.005*"оказываться" + 0.005*"ночь" + 0.005*"берег" + 0.004*"остров" + '
  '0.004*"корабль" + 0.004*"вода"'),
 (4,
  '0.009*"отец" + 0.008*"дом" + 0.008*"артур" + 0.007*"сын" + 0.007*"мать" + '
  '0.006*"девушка" + 0.006*"доктор" + 0.006*"семья" + 0.006*"рудольф" + '
  '0.005*"мистер"'),
 (5,
  '0.008*"квентин" + 0.007*"книга" + 0.007*"профессор

### Compute Model Perplexity and Coherence Score

In [34]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['text_pymystem_list'],
                                     dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.03169675114182

Coherence Score:  0.4343248563227674
CPU times: user 27.7 s, sys: 30.7 s, total: 58.4 s
Wall time: 1min 19s


### Visualize the topics-keywords

In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
14     24.214140  1        1       0.246830  0.016443
16     13.091763  1        2       0.244951  0.027140
8      9.882092   1        3       0.195093  0.124185
13     9.389684   1        4       0.208218 -0.084295
3      8.012460   1        5       0.158306  0.074224
18     6.969986   1        6       0.143247 -0.259522
7      6.760411   1        7       0.163733 -0.229430
4      3.947788   1        8       0.133620  0.021296
12     2.967502   1        9      -0.021942  0.077348
2      2.665565   1        10     -0.045390  0.080041
6      1.984158   1        11      0.086191  0.194647
10     1.762157   1        12     -0.129100  0.014610
15     1.661567   1        13     -0.031585  0.122987
5      1.411659   1        14     -0.168456 -0.064690
0      1.191142   1        15     -0.157648  0.069527
19     1.153518   1        16     -0.193453 -0.085274
9      0.951921   1        17     -0.186149 -0.049510
1      0.889621   1        18     -0.202357  0.006539
11     0.568793   1        19     -0.212553 -0.014098
17     0.524074   1        20     -0.231560 -0.042167, topic_info=      Category         Freq           Term        Total  loglift  logprob
term                                                                     
41183  Default  7511.000000  человек        7511.000000  30.0000  30.0000
22775  Default  3713.000000  отец           3713.000000  29.0000  29.0000
47723  Default  2758.000000  жена           2758.000000  28.0000  28.0000
2149   Default  2987.000000  сын            2987.000000  27.0000  27.0000
11689  Default  3697.000000  дом            3697.000000  26.0000  26.0000
9042   Default  2486.000000  мать           2486.000000  25.0000  25.0000
41938  Default  2145.000000  дочь           2145.000000  24.0000  24.0000
2731   Default  608.000000   нильс          608.000000   23.0000  23.0000
56449  Default  1153.000000  бог            1153.000000  22.0000  22.0000
8913   Default  3851.000000  жизнь          3851.000000  21.0000  21.0000
57950  Default  2339.000000  девушка        2339.000000  20.0000  20.0000
43202  Default  1487.000000  король         1487.000000  19.0000  19.0000
36983  Default  1437.000000  герой          1437.000000  18.0000  18.0000
8926   Default  1907.000000  муж            1907.000000  17.0000  17.0000
50411  Default  1990.000000  любовь         1990.000000  16.0000  16.0000
50123  Default  1395.000000  семья          1395.000000  15.0000  15.0000
46543  Default  1267.000000  война          1267.000000  14.0000  14.0000
22730  Default  2368.000000  говорить       2368.000000  13.0000  13.0000
41776  Default  606.000000   невеста        606.000000   12.0000  12.0000
51104  Default  1594.000000  убивать        1594.000000  11.0000  11.0000
1162   Default  1006.000000  парцифаль      1006.000000  10.0000  10.0000
9114   Default  1681.000000  деньги         1681.000000  9.0000   9.0000 
47794  Default  1383.000000  работа         1383.000000  8.0000   8.0000 
20314  Default  590.000000   вера           590.000000   7.0000   7.0000 
6163   Default  3321.000000  год            3321.000000  6.0000   6.0000 
8221   Default  671.000000   царь           671.000000   5.0000   5.0000 
55047  Default  1929.000000  молодой        1929.000000  4.0000   4.0000 
26467  Default  1065.000000  работать       1065.000000  3.0000   3.0000 
49744  Default  881.000000   компания       881.000000   2.0000   2.0000 
17782  Default  2021.000000  ребенок        2021.000000  1.0000   1.0000 
...        ...         ...        ...              ...      ...      ... 
50187  Topic20  74.477936    иосел          75.351242    5.2396  -4.4123 
32404  Topic20  50.445740    спейд          51.319046    5.2341  -4.8020 
23620  Topic20  46.854725    анюта          47.728031    5.2328  -4.8758 
16546  Topic20  45.863258    рейхард        46.736565    5.2324  -4.8972 
37125  Topic20  44.396004    па